<a href="https://colab.research.google.com/github/hstrangis/7506R_2C2022_GRUPO4_N2/blob/main/7506R_TP2_GRUPO4_ENTREGA_N2%2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo práctico 2: Propiedades en venta
# Grupo 4

### Integrantes
### 106963	- Matias Daniel Rueda
### 96407 - Nicolás Hernán Strangis
### 93525 - Miguel Centellas Leon

# Imports y setup

Importación e instalación de librerías y paquetes necesarios

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import geopandas as gpd
import seaborn as sns
import numpy as np
import folium
import matplotlib
import mapclassify
import joblib
import re

from sklearn.model_selection import train_test_split

In [32]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [253]:
df_test = pd.read_csv("/content/drive/MyDrive/datasets/df_test_post_preprocesamiento.csv")
df_train = pd.read_csv("/content/drive/MyDrive/datasets/df_train_post_preprocesamiento.csv")
df_original = pd.read_csv("/content/drive/MyDrive/datasets/properati_argentina_2021.csv")

In [252]:
df_descrip = pd.read_csv("/content/drive/MyDrive/datasets/properati_argentina_2021_decrip.csv", engine='python', encoding='utf-8', error_bad_lines=False)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


#a) Ampliación del dataset

In [254]:
df_descrip.head()

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!<br>APTO ...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...


Eliminamos filas que contienen NaN del dataset de descripciones, para no tener problemas posteriormente al ejecutar ECMes

In [255]:
df_descrip = df_descrip.dropna()

Verificamos que no hay descripciones vacias

In [256]:
df_descrip.loc[df_descrip['property_description'].apply(type) != str].sum()

id                      0.0
property_description    0.0
dtype: float64

Quitamos caracteres que luego nos traeran problemas con ECMes

In [257]:
df_descrip['property_description'] = df_descrip['property_description'].replace(['{','}','\[',']','\(','\)'],'', regex=True)
df_descrip['property_description'] = df_descrip['property_description'].replace('<br>','\n', regex=True)
df_descrip.head(10)

,id,property_description
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!\nAPTO CR...
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,..."
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...
5,XztoBkv1vE+dGuId8GiPyA==,"Se Vende PH interno, ubicado en excelente barr..."
6,JEhUB1WqHYnvkmuPWlDgmg==,La propiedad se encuentra ubicada en Gobernado...
7,ucluEQ4sNKNVbwyKJFnaBw==,Casa en venta sobre calle Alejandro Magno con ...
8,a/LC2bANkFuEJYuHglYSTg==,Hermosa Casa en Ph ubicado a 10 cuadras del ce...
9,XWCMzAub+1YSpvBCuREdPA==,Hermosa casa en PH ubicada en barrio Santa Rit...


Generamos un archivo txt, que contiene las descripciones (solo 100, ya que sino demora muchisimo, fuimos tomando varias muestras aleatorias) sin nulos y sin caracteres que generen conflictos con las expresiones regulares

In [258]:
with open("descripciones.txt", "w") as file_out:
  file_out.write('\n'.join(df_descrip['property_description'].head(100)))

Este archivo es procesado con ECMes localmente, obteniendo relaciones que nos permitieron extraer palabras claves para generar nuevas columnas de datos

Una vez analizados los resultados obtenidos, decidimos crear las siguientes columnas, utilizando como herramienta expresiones regulares, findall especificamente

Columna nueva, indica si el aviso posee pileta/piscina

In [259]:
df_descrip['tiene_pileta'] = df_descrip['property_description'].str.findall('pileta | piscina', flags=re.IGNORECASE)
df_descrip['tiene_pileta'] = np.where(df_descrip['tiene_pileta'], 1, 0)

Columna nueva, indica si el aviso posee parrilla

In [260]:
df_descrip['tiene_parrilla'] = df_descrip['property_description'].str.findall('parrilla', flags=re.IGNORECASE)
df_descrip['tiene_parrilla'] = np.where(df_descrip['tiene_parrilla'], 1, 0)

Columna nueva, indica si el aviso posee cochera

In [261]:
df_descrip['tiene_cochera'] = df_descrip['property_description'].str.findall('cochera', flags=re.IGNORECASE)
df_descrip['tiene_cochera'] = np.where(df_descrip['tiene_cochera'], 1, 0)

Columna nueva, indica si el aviso posee terraza/balcón

In [262]:
df_descrip['tiene_terraza'] = df_descrip['property_description'].str.findall('terraza|balcon', flags=re.IGNORECASE)
df_descrip['tiene_terraza'] = np.where(df_descrip['tiene_terraza'], 1, 0)

Columna nueva, indica si el aviso posee o no expensas, o si son bajas ('Sin expensas', 'Bajas expensas' o 'Con expensas')

Columna que identifica todos los avisos que contienen alguna descripcion vinculada a las expensas

In [263]:
df_descrip['con_expensas'] = df_descrip['property_description'].str.findall('expensas | expensa', flags=re.IGNORECASE)
df_descrip['con_expensas'] = np.where((df_descrip['con_expensas'].str.len() != 0), 'Con expensas', '')

Luego creamos una columna que identifica los avisos con "Bajas" expensas

In [264]:
df_descrip['bajas_expensas'] = df_descrip['property_description'].str.findall('bajas expensas|expensas bajas', flags=re.IGNORECASE)
df_descrip['bajas_expensas'] = np.where((df_descrip['bajas_expensas'].str.len() != 0), 'Bajas expensas', '')

Finalmente, una que contiene los avisos de lugares "Sin" expensas

In [265]:
df_descrip['sin_expensas'] = df_descrip['property_description'].str.findall('sin expensas|expensas $0|expensas $ 0', flags=re.IGNORECASE)
df_descrip['sin_expensas'] = np.where((df_descrip['sin_expensas'].str.len() != 0), 'Sin expensas', '')

Por ultimo concatenamos las 3 nuevas columnas, y dejamos la celda vacia ('') en caso de que no se haga referencia a las expensas 

In [266]:
df_descrip['expensas'] = df_descrip['bajas_expensas'] + df_descrip['sin_expensas']

# Se concatena solo en los casos donde hay '', ya que sino se sobreescribirian los demás valores
df_descrip['expensas'] = df_descrip[['expensas','con_expensas']].sum(axis=1).where(df_descrip['expensas'] == '', df_descrip['expensas'])

Eliminamos las columnas de expensas intermedias a la resultante 'expensas'

In [267]:
df_descrip.drop(['con_expensas', 'sin_expensas', 'bajas_expensas'], axis = 'columns', inplace=True)
df_descrip.head(10)

,id,property_description,tiene_pileta,tiene_parrilla,tiene_cochera,tiene_terraza,expensas
0,BZCeiGkJr1WBUi6pKJQwJQ==,Corredor Responsable: MARIA ALEJANDRA GENOVEVA...,0,0,0,1,Bajas expensas
1,zuHOGgUE0UB71W7m/YCLoA==,¡HERMOSO PH TIPO CASA DE 2 AMBIENTES!\nAPTO CR...,0,0,0,0,Sin expensas
2,TUL813tXchVBlHUsfaA6DA==,Excelente PH en Núñez 4 amb!!!! Oportunidad !!...,0,1,0,1,
3,gn55JPmcjftBV/at1a7fPg==,"UNICO PH DE 4 DORMITORIOS EN IMPECABLE ESTADO,...",0,0,1,1,Sin expensas
4,qg4m3QHcuo1+wMmyp7kx3Q==,Ph 2 Amb. Villa Crespo exclusivo uso profesion...,0,0,0,0,
5,XztoBkv1vE+dGuId8GiPyA==,"Se Vende PH interno, ubicado en excelente barr...",0,0,1,0,
6,JEhUB1WqHYnvkmuPWlDgmg==,La propiedad se encuentra ubicada en Gobernado...,1,0,1,0,
7,ucluEQ4sNKNVbwyKJFnaBw==,Casa en venta sobre calle Alejandro Magno con ...,1,0,1,0,
8,a/LC2bANkFuEJYuHglYSTg==,Hermosa Casa en Ph ubicado a 10 cuadras del ce...,0,0,0,1,
9,XWCMzAub+1YSpvBCuREdPA==,Hermosa casa en PH ubicada en barrio Santa Rit...,0,0,0,0,


Para agregar las columnas nuevas al df preprocesado, joineamos las columnas del dataset de descripciones usando como clave la columna 'id'

In [268]:
df_ampliado_train = pd.merge(df_train, df_descrip[['tiene_cochera','tiene_parrilla','tiene_pileta','tiene_terraza','expensas','id']], on='id')
df_ampliado_test = pd.merge(df_test, df_descrip[['tiene_cochera','tiene_parrilla','tiene_pileta','tiene_terraza','expensas','id']], on='id')

# Borramos columna innecesaria
df_ampliado_train.drop(['Unnamed: 0'], axis = 'columns', inplace=True)
df_ampliado_test.drop(['Unnamed: 0'], axis = 'columns', inplace=True)
df_ampliado_train.head(100)

,id,latitud,longitud,place_l2,place_l3,operation,property_type,property_rooms,property_bedrooms,property_surface_total,property_surface_covered,property_price,property_currency,property_title,tiene_cochera,tiene_parrilla,tiene_pileta,tiene_terraza,expensas
0,VZsvT1lQEuvPqEM4cjT8/g==,-34.566536,-58.479638,Capital Federal,Coghlan,Venta,Departamento,2.0,1.0,65.0,53.0,198000.0,USD,1 - Holmberg 2700,1,1,1,1,
1,QlFGJ8qbIENvmMXqW4JbsQ==,-34.645625,-58.459408,Capital Federal,Flores,Venta,Departamento,1.0,1.0,29.0,29.0,63000.0,USD,Departamento a estrenar - Edificio con ameniti...,0,1,1,0,
2,D9nGcyK9B6vsyeUvuT8qbQ==,-34.592460,-58.453428,Capital Federal,Caballito,Venta,Departamento,2.0,1.0,41.0,35.0,93000.0,USD,Departamento de 2 ambientes con balcón aterraz...,0,0,0,1,Con expensas
3,FLCnXUUffcnknzXIH9pBCg==,-34.577033,-58.423367,Capital Federal,Palermo,Venta,Departamento,1.0,1.0,49.0,44.0,160000.0,USD,Departamento - Monoambiente - Palermo Nuevo - ...,0,1,1,1,
4,U1B7lvnYtd4L18Ccb8mvhw==,-34.580210,-58.412815,Capital Federal,Palermo,Venta,Departamento,1.0,2.0,113.2,78.0,290000.0,USD,Departamento en Palermo,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CmzBSYbHVmyVPDKlI+MA7w==,-34.592927,-58.443849,Capital Federal,Villa Crespo,Venta,Departamento,2.0,1.0,30.0,30.0,69900.0,USD,DARWIN 700 - T/SOL - Villa Crespo,0,0,0,0,
96,uV8HYBPZ40y2IQUm0zCoHw==,-34.543026,-58.473482,Capital Federal,Saavedra,Venta,Departamento,2.0,1.0,35.0,35.0,77000.0,USD,VENTA 2 AMBIENTES OPORTUNIDAD LUZ SAAVEDRA TER...,0,1,0,1,Con expensas
97,QTHzIXzZylNJSpu8yKIemw==,-34.604960,-58.448139,Capital Federal,Villa Crespo,Venta,PH,2.0,1.0,30.0,24.0,60000.0,USD,PH MUY LUMINOSO-2 AMB - PB - VILLA CRESPO,0,0,0,0,Con expensas
98,pt4YRpYfASBQQXT3D9tBpw==,-34.596886,-58.403090,Capital Federal,Barrio Norte,Venta,Departamento,3.0,2.0,74.0,71.0,169000.0,USD,Departamento 3 ambientes con balcón A ESTRENAR...,0,0,0,0,


##Guardamos el dataset ampliado

In [270]:
df_ampliado_train.to_csv("df_ampliado_train.csv");
df_ampliado_test.to_csv("df_ampliado_test.csv");